<a href="https://colab.research.google.com/github/domschl/transformer-poet/blob/main/transformer_poet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformer-Poet

Please review [ml-indie-tools](https://github.com/domschl/ml-indie-tools), a collection machine learning tools that provides support for more environment indepent code. It will access your Google Drive when using with Google Colab.

In [1]:
!pip install -U ml-indie-tools

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import logging
import os
import sys
import copy
import json
import time
import datetime
import random

import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, regularizers

import tensorflow_datasets as tfds

In [3]:
from ml_indie_tools.env_tools import MLEnv
from ml_indie_tools.Gutenberg_Dataset import Gutenberg_Dataset
from ml_indie_tools.Text_Dataset import Text_Dataset

from ml_indie_tools.keras_custom_layers import MultiHeadSelfAttention, PositionalEncoding

Using TF-Keras version: 2.9.0


## Preliminary

A tensorflow deep multi-head attention model for text generation

This code can use either CPU, GPU, TPU when running on Google Colab.
Select the corresponding runtime (menu: **`Runtime / Change runtime type`**)

## 0. Environment

In [4]:
cached_batch_data = None   # Do regenerate time-consuming training data, if aleady cached.

ml_env = MLEnv(platform='tf', accelerator='fastest')
ml_env.describe()

'OS: Linux, Python: 3.7.15, Colab Jupyter Notebook Tensorflow: 2.9.2, GPU: Tesla T4 (3MiB / 15109MiB)'

In [5]:
use_eager=tf.executing_eagerly()
if ml_env.is_tpu is True:
    tpu_strategy = ml_env.tpu_strategy
    tpu_is_init=True
    if use_eager is True:
        tf.config.run_functions_eagerly(False)
    use_eager=False

In [6]:
project_name='women_writers'
model_name='mhsa_v1_tf'

# NOTICE: This will request access to Google Drive, if running on Google Colab. Google Drive is used to store snapshots
# training data. See project ml-indie-tools: https://github.com/domschl/ml-indie-tools 
#
# Note: you need to allow popups in your browser for COLAB, otherwise you won't see the google-drive login box, and drive access will fail!

root_path, project_path, model_path, data_path, log_path = ml_env.init_paths(project_name=project_name, model_name=model_name)

print(f"Root path (all projects) : {root_path} (This will be '.' (current dir) for local projects, and a google drive path for Colab)")
print(f"Project path             : {project_path} (Changes to the file system happen only below this project path")
print(f"Model path (snapshots)   : {model_path} (Model weights and snapshots are stored here)")
print(f"Data path (training data): {data_path} (Training data will be downloaded here)")
print(f"Log dir (tensorboard)    : {log_path} (it doesn't work to put logs on gdrive due to caching, hence local dir)")

Mounted at /content/drive
Root path (all projects) : /content/drive/My Drive (This will be '.' (current dir) for local projects, and a google drive path for Colab)
Project path             : /content/drive/My Drive/Colab Notebooks/women_writers (Changes to the file system happen only below this project path
Model path (snapshots)   : /content/drive/My Drive/Colab Notebooks/women_writers/model/mhsa_v1_tf (Model weights and snapshots are stored here)
Data path (training data): /content/drive/My Drive/Colab Notebooks/women_writers/data (Training data will be downloaded here)
Log dir (tensorboard)    : ./logs (it doesn't work to put logs on gdrive due to caching, hence local dir)


##  1. Text library

`Text_Dataset` and `Gutenberg_Dataset` classes: libraries for training, 
encoding, batch generation, and formatted source display. It read some 
books from Project Gutenberg and supports creation of training batches. 
The output functions support highlighting to allow to compare generated 
texts with the actual sources to help to identify identical (memorized) 
parts.

In [7]:
use_dark_mode=False # Set to false for white background. HTML-text-compare uses background-colorization to identify different sources. Those background colors are dependent on the theme type.

In [8]:
logging.basicConfig(level=logging.INFO)
cache_dir = os.path.join(data_path, 'gutenberg_cache')
gd = Gutenberg_Dataset(cache_dir=cache_dir)

In [9]:
# sample searches
search_spec= {"author": ["Emily Brontë", "Jane Austen", "Virginia Woolf"], "language": ["english"]}

book_list=gd.search(search_spec)
book_cnt = len(book_list)
print(f"{book_cnt} matching books found with search {search_spec}.")
if book_cnt<40:
    # Note: please verify that book_cnt is 'reasonable'. If you plan to use a large number of texts, 
    # consider [mirroring Gutenberg](https://github.com/domschl/ml-indie-tools#working-with-a-local-mirror-of-project-gutenberg)
    book_list = gd.insert_book_texts(book_list, download_count_limit=book_cnt)  
else:
    logging.error("Please verify your book_list, a large number of books is scheduled for download. ABORTED.")

21 matching books found with search {'author': ['Emily Brontë', 'Jane Austen', 'Virginia Woolf'], 'language': ['english']}.


In [10]:
for i in range(len(book_list)):
    print(f"{i}: {book_list[i]['title']} - {book_list[i]['author']}, {book_list[i]['ebook_id']}")

0: The Common Reader - Virginia Woolf, 64457
1: Mr. Bennett and Mrs. Brown - Virginia Woolf, 63022
2: The Younger Sister, Volumes 1-3 - Catherine Anne Austen Hubback and Jane Austen, 54066
3: The Younger Sister, Vol. 3 - Catherine Anne Austen Hubback and Jane Austen, 54012
4: The Younger Sister, Vol. 2 - Catherine Anne Austen Hubback and Jane Austen, 54011
5: The Younger Sister, Vol. 1 - Catherine Anne Austen Hubback and Jane Austen, 54010
6: Pride and Prejudice - Jane Austen, 42671
7: The Letters of Jane Austen - Jane Austen, 42078
8: The Complete Project Gutenberg Works of Jane Austen - Jane Austen, 31100
9: Jacob's Room - Virginia Woolf, 5670
10: Pride and Prejudice - Jane Austen, 1342
11: Night and Day - Virginia Woolf, 1245
12: Love And Friendship And Other Early Works - Jane Austen, 1212
13: Lady Susan - Jane Austen, 946
14: Wuthering Heights - Emily Brontë, 768
15: Sense and Sensibility - Jane Austen, 161
16: Emma - Jane Austen, 158
17: The Voyage Out - Virginia Woolf, 144
18: M

In [11]:
MAX_TOKENS = 20000  # This becomes vocab_size
MAX_NGRAM_LEN = 8   # Max length of a token

select = ("Bennett", "1342", "5670", "1245", "161", "141", "121", "105", "Susan", "Wuthering", "Emma", "Voyage")  # List unique single-words from title or ebook_id to select a given book
sub_book_list = [book_list[i] for i in range(len(book_list)) if not set([book_list[i]['ebook_id']]+book_list[i]['title'].split(' ')).isdisjoint(set(select))]

print("Using:")
for i in range(len(sub_book_list)):
    print(f"{i+1}: {sub_book_list[i]['title']} - {sub_book_list[i]['author']}")

textlib_dataset = None  # Forces re-caching
td = Text_Dataset(sub_book_list)
td.init_tokenizer(tokenizer='ngram', max_ngrams=MAX_NGRAM_LEN, max_tokens=MAX_TOKENS)


Using:
1: Mr. Bennett and Mrs. Brown - Virginia Woolf
2: Jacob's Room - Virginia Woolf
3: Pride and Prejudice - Jane Austen
4: Night and Day - Virginia Woolf
5: Lady Susan - Jane Austen
6: Wuthering Heights - Emily Brontë
7: Sense and Sensibility - Jane Austen
8: Emma - Jane Austen
9: The Voyage Out - Virginia Woolf
10: Mansfield Park - Jane Austen
11: Northanger Abbey - Jane Austen
12: Persuasion - Jane Austen


In [12]:
SEQUENCE_LEN = 80
SUB_PROBABILITY = 0.15  # like BERT

td.init_getitem(sample_type='encoded', sample_length=SEQUENCE_LEN, content_stepping=1)

num_records = len(td)

print(f"{num_records} records")

1529756 records


In [13]:
def get_sample_batch(td, batch_size, length, SUB_probability=0.15):
    for i in range(batch_size):
        Xi = td.get_random_item()
        yi = Xi.copy()
        l=int(len(Xi)*SUB_probability)
        for li in range(l):
            pos=random.randint(0,len(Xi)-1)
            if td.tokenizer_type=='char':
                Xi[pos]=td.c2i['␚']
            elif td.tokenizer_type=='word':
                Xi[pos]=td.w2i['<subst>']
            elif td.tokenizer_type=='ngram':
                Xi[pos]=td.t2i['<subst>']
            else:
                print(f"Unexpected tokenizer_type {td.tokenizer_type}")
        if i==0:
            # smpX=np.array(Xi, dtype=np.float32)
            smpX=np.array(Xi, dtype=np.int32)
            smpy=np.array(yi, dtype=np.int32)
        else:
            # smpX = np.vstack((smpX, np.array(Xi, dtype=np.float32)))
            smpX = np.vstack((smpX, np.array(Xi, dtype=np.int32)))
            smpy = np.vstack((smpy, np.array(yi, dtype=np.int32)))
    return np.array(smpX), np.array(smpy)

In [14]:
test_x, test_y = get_sample_batch(td, 2, 40, SUB_probability=SUB_PROBABILITY)
for i in range(len(test_x)):
    xi=[int(x) for x in test_x[i]]
    print(f"[{i}](l={len(xi)}): X=>{td.decode(xi)}<, y=>{td.decode(test_y[i])}<")

[0](l=80): X=>alua<subst>substance which she still called
reality, and still <subst> that she could find. The Hilbery<subst>s the
saying is, “knew <subst>one,<subst>and that arrogant claim was certainly
uph<subst>by the number of houses which, within a certain area, lit their
lamps at night<subst>pened their doors after <subst> p<subst>m., and admitted the
Hilberys to the<subst>dining-rooms, s<subst>once a mont<, y=>aluable substance which she still called
reality, and still believed that she could find. The Hilberys, as the
saying is, “knew every one,” and that arrogant claim was certainly
upheld by the number of houses which, within a certain area, lit their
lamps at night, opened their doors after 3 p. m., and admitted the
Hilberys to their dining-rooms, say, once a mont<
[1](l=80): X=>e,
      was along the open grove which <subst>d that side of the park, where
      t<subst> a nice sheltered <subst>h, which no one seemed to value but
      herself, and where she <subst>beyond the 

In [15]:
test_x.shape, test_y.shape

((2, 80), (2, 80))

## 2. Use tf.data for texts

In [16]:
def expand_name_template(template, params):
    exp=copy.copy(template)
    for key in params:
        src="{"+key+"}"
        dst=f"{params[key]}"
        exp=exp.replace(src,dst).replace('[','(').replace(']',')')
    return exp

def save_model_metadata(epoch, suffix='std'):
    meta_file = os.path.join(model_path, f'model_meta_{suffix}.json')
    params['current_epoch'] = epoch
    try:
        with open(meta_file, 'w') as f:
            f.write(json.dumps(params))
    except Exception as e:
        print(f"Failed to store model metadata at {model_path}: {e}")
        return False
    return True

def read_model_metadata(suffix="std"):
    meta_file = os.path.join(model_path, f'model_meta_{suffix}.json')
    try:
        with open(meta_file, 'r') as f:
            meta = json.load(f)
    except Exception as e:
        print(f"Cannot access project meta-data at {meta_file}: {e}, starting anew.")
        return None
    return meta

def is_metadata_compatible(params, meta):
    is_valid=True
    keys=set(list(params.keys())+list(meta.keys()))
    for key in keys:
        if key in updatable_keys:
            continue
        if key not in meta:
            print(f"Key {key} not available in last checkpoint model_meta, params[{key}]: {params[key]}, cannot import incompatible model. Put key in `updatable_keys` list, if irrelevant.")
            is_valid = False
        elif key not in params:
            print(f"Key {key} not available in params, last checkpoint model_meta[{key}]: {meta[key]}, cannot import incompatible model. Put key in `updatable_keys` list, if irrelevant.")
            is_valid = False
        elif meta[key]!=params[key]:
            print(f"Last checkpoint model_meta[{key}]: {meta[key]} != params[{key}]: {params[key]}, cannot import incompatible model. Put key in `updatable_keys` list, if irrelevant.")
            is_valid = False
    if is_valid is False:
        print("Aborting import.")
        return False
    return True

In [17]:
vocabulary_size = td.get_unique_token_count()  # vocabulary-size

lyrs = 8;

params = { # Multi-head self-attention
    'name': '{mhsa_layers}x{heads}x{units}x{vocab_size}',

    'mhsa_layers': lyrs, 
    'heads': [12]*lyrs,
    'units': [512]*lyrs,  # 0 inserts an LSTM for memory-states :-)
    'norm': 'softmax', # this is for within each head
    'mh_normalize': True,  # use layer-norm after concatenation (or additiona) of the heads
    'l2_regularizer': 1e-9,
    'dropout': 0.0,       # no dropout: 0.0
    'join_heads_by_add': True,  # stragegy how multi-heads are joined: False: concat (as in all-you-need), True: relu+add of all the heads
    'vocab_size': vocabulary_size,
    'sequence_len': SEQUENCE_LEN,
    'embedding_size': 128,

    'batch_size': 256,
    'learning_rate': 0.00002,
    'clipvalue': None,
    'sample_every_n_epochs': 100,
}

if len(params['heads'])!=params['mhsa_layers'] or len(params['units'])!=params['mhsa_layers']:
    print("ERROR: lenght of 'heads' and 'units' must be equal to mhsa_layers!")
    
if ml_env.is_tpu is True:
    lr = params['learning_rate']*1.0
else:
    lr = params['learning_rate']

model_suffix = expand_name_template(params['name'], params)
# Put 'important' params in checkpoint-pathname to separate model-data:
checkpoint_dir = os.path.join(model_path, f"training_checkpoints_{model_suffix}")
if os.path.exists(checkpoint_dir) is False:
    os.makedirs(checkpoint_dir)

# When comparing if training-data is compatible with new params set, 
# the following keys are updatable, they can be changed while continuing
# to use existing checkpoints and continue training with those values
# changed:
updatable_keys=['learning_rate', 'batch_size', 'current_epoch', 'dropout', 
             'sample_every_n_epochs']

# These values are taking from saved checkpoint:
keep_keys=['current_epoch']

continue_last = True
if continue_last is False:
    print("NOT continuing based on existing training! New start.")

meta = read_model_metadata(suffix=model_suffix)
if meta is not None and is_metadata_compatible(params, meta) is True and continue_last is True:
    for key in keep_keys:
        if key in meta:
            params[key]=meta[key]
    if params is not None:
        print(f"Continuing last session from epoch {params['current_epoch']}")
    else:
        print(f"No previous data, starting new model")
else:
    print("Starting new model")

print(params)

Continuing last session from epoch 16823
{'name': '{mhsa_layers}x{heads}x{units}x{vocab_size}', 'mhsa_layers': 8, 'heads': [12, 12, 12, 12, 12, 12, 12, 12], 'units': [512, 512, 512, 512, 512, 512, 512, 512], 'norm': 'softmax', 'mh_normalize': True, 'l2_regularizer': 1e-09, 'dropout': 0.0, 'join_heads_by_add': True, 'vocab_size': 20000, 'sequence_len': 80, 'embedding_size': 128, 'batch_size': 256, 'learning_rate': 2e-05, 'clipvalue': None, 'sample_every_n_epochs': 100, 'current_epoch': 16823}


In [18]:
num_batches = num_records // params['batch_size']
print(f"num_batches = {num_batches}")

num_batches = 5975


In [19]:
# @tf.function   (only slows things down [considerably!])
def make_tf_dataset(num, random_index=False, SUB_probability=0.0):
    dx=[]
    dy=[]
    num_batches_active = num
    for i in range(num_batches_active):
        x,y=get_sample_batch(td, params['batch_size'], params['sequence_len'], SUB_probability=SUB_probability)
        if i<1:
            print(f"[{num} x]: {x.shape} -> {y.shape}")
        dx.append(x)
        dy.append(y)
    dx=np.array(dx)
    dy=np.array(dy)
    print(f"dx.shape={dx.shape}, dy.shape={dy.shape}")
    data_xy = (dx, dy)
    tf_dataset=tf.data.Dataset.from_tensor_slices(data_xy)
    return tf_dataset

In [22]:
MAX_NUM_BATCHES = 50000

if num_batches>MAX_NUM_BATCHES:
    restricted_batches=MAX_NUM_BATCHES
    print(f"Restrictinig {num_batches} to max of {restricted_batches}")
else:
    restricted_batches=num_batches
    print(f"{restricted_batches} batches")
if cached_batch_data == restricted_batches and textlib_dataset is not None:
    print("Reusing cached training-data")
else:
    print("Creating dataset, this is slow. Be patient...")
    textlib_dataset = make_tf_dataset(restricted_batches, SUB_probability=SUB_PROBABILITY)
    cached_batch_data = restricted_batches
    print("Dataset done and cached.")

5975 batches
Creating dataset, this is slow. Be patient...
[5975 x]: (256, 80) -> (256, 80)
dx.shape=(5975, 256, 80), dy.shape=(5975, 256, 80)
Dataset done and cached.


In [23]:
shuffle_buffer=10000
if ml_env.is_tpu is True:
    dataset=textlib_dataset.shuffle(shuffle_buffer).repeat()  # Otherwise TPU may run dry
else:
    dataset=textlib_dataset.shuffle(shuffle_buffer)  
dataset.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(256, 80), dtype=tf.int32, name=None), TensorSpec(shape=(256, 80), dtype=tf.int32, name=None))>

In [24]:
if ml_env.is_tpu is False:
    validation_dataset = make_tf_dataset(10, random_index=True, SUB_probability=SUB_PROBABILITY)

In [20]:
def model_mhsa(inputs, params):
    dense = layers.Dense(params['vocab_size'], kernel_regularizer=regularizers.l2(params['l2_regularizer']))  # using softmax here prevents temperature adjust, affects 'from_logits' param in sparse_categorical loss 
    fl = layers.Flatten()
    dr = layers.Dropout(params['dropout'])
    pe = PositionalEncoding(amplitude=0.3)
    rs_up = layers.Reshape(target_shape=(SEQUENCE_LEN,vocabulary_size))
    if 0 in params['units']:
        lstm1 = layers.LSTM(units=vocabulary_size, return_sequences=True)
    if vocabulary_size>=300:
        emb=layers.Embedding(vocabulary_size,params['embedding_size'],input_length=params['sequence_len'])
    rs_down = layers.Reshape(target_shape=(SEQUENCE_LEN,vocabulary_size))
    mhsa=[]
    residuals=[]

    for i in range(params['mhsa_layers']):
        if params['units'][i]==0:
            mhsa.append(None)
            residuals.append(i)
        else:
            mhsa.append(MultiHeadSelfAttention(params['heads'][i], units=params['units'][i], norm=params['norm'], mh_normalize=params['mh_normalize'], join_heads_by_add=params['join_heads_by_add']))
    xint = tf.cast(inputs,dtype=tf.int32)
    if vocabulary_size<300:
        x = tf.one_hot(xint, params['vocab_size'], axis=-1)
    else:
        x = emb(xint)
    x = pe(x)
    for i in range(len(mhsa)):
        if i in residuals:
            x = rs_down(lstm1(rs_up(x)))+x
            print(f"Residual at layer {i} added.")
        else:
            x = mhsa[i](x)
        # x = mhsa[i](x,x)
    if params['dropout']>0.0:
        x = dr(x)
    # x = dense(fl(x))
    x = dense(x)
    return x 

In [21]:
def mhsa_generate(model, text, gen_len=64, temperature=0.9, argmax=False, verbose=False):
    if verbose is True:
        full=text[:-1]
    gen_text=""
    lf=0
    input = np.array(td.encode(text))
    while len(input) < params['sequence_len']:
        input = np.concatenate([td.encode('<pad>'),input])
    for i in range(gen_len):
        input = np.concatenate([input[1:],td.encode('<subst>')])
        if len(input)!=params['sequence_len']:
            print('assertion failure')
            return None
        pred = model(input)
        pred /= temperature
        pred = tf.keras.layers.Softmax()(pred)
        if tf.executing_eagerly() is True and ml_env.is_tpu is False:
            pred=pred.numpy()
        else:
            pred=tf.keras.backend.eval(pred)  # this is a cheat, it internaly used Numpy() too.
        if argmax is True:
            pred=np.argmax(pred[0],axis=1)
        else:
            pred = [np.random.choice(list(range(len(pred[0][-1]))), p=pred[0][-1])]
        input = np.concatenate([input[1:],[pred[-1]]])
        c = td.decode([pred[-1]])
        if verbose is True:
            print(c, end='')
            if c=='\n':
                lf=0
            else:
                lf += 1
                if (lf>80 and c==' ') or lf>120:
                    print()
                    lf=0
            full+=c
        gen_text+=c
    if verbose is True:
        print()
    return gen_text


In [22]:
if ml_env.is_tpu is True:
    with tpu_strategy.scope():
        print("Creating TPU-scope model")
        inputs = keras.Input(shape=(params['sequence_len'],))
        outputs = model_mhsa(inputs, params)
        model = keras.Model(inputs=inputs, outputs=outputs, name="mhsa_v1_tf")
    print("Creating Default-scope model")
    inputs = keras.Input(shape=(params['sequence_len'],))
    outputs = model_mhsa(inputs, params)
    model_cpu = keras.Model(inputs=inputs, outputs=outputs, name="mhsa_v1_tf")
else:
    inputs = keras.Input(shape=(params['sequence_len'],))
    outputs = model_mhsa(inputs, params)
    model = keras.Model(inputs=inputs, outputs=outputs, name="mhsa_v1_tf")
    model_cpu = model

In [23]:
def get_newest_checkpoint(checkpoint_dir):
    files = os.listdir(checkpoint_dir)
    paths = [os.path.join(checkpoint_dir, basename) for basename in files]
    return max(paths, key=os.path.getctime)

def import_previous_compatible_checkpoint(model, force_import=False):
    meta = read_model_metadata(suffix=model_suffix)
    if meta is None:
        print("No previous checkpoint found")
        return False
    if is_metadata_compatible(params, meta) is not True and force_import is False:
        print("No useable import found.")
        return False
    try:
        last_checkpoint = get_newest_checkpoint(checkpoint_dir) # Doesn't do anything: tf.train.latest_checkpoint(checkpoint_dir)
    except Exception as e:
        print(f"Cannot determine last checkpoint in {checkpoint_dir}, cannot import due to: {e}")
        return False
    print(f"Last checkpoint: {last_checkpoint}")
    try:
        model.load_weights(last_checkpoint)
    except Exception as e:
        print(f"Failed to import model {last_checkpoint}: {e}")
        return False
    if 'current_epoch' in meta:
        params['current_epoch'] = meta['current_epoch']
    print(f"Successful import of epoch {params['current_epoch']} from {last_checkpoint}, continuing from there...")
    return True

### Loss function, optimizer, tensorboard output

In [39]:
kscc = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.NONE)

def loss(labels, logits):
  vl=kscc(labels, logits)
  return vl

In [40]:
if params['clipvalue'] is not None:
    if ml_env.is_tpu is True:
        with tpu_strategy.scope():
            opti = tf.keras.optimizers.Adam(learning_rate=lr, clip_value=params['clipvalue'])
    else:
        opti = tf.keras.optimizers.Adam(learning_rate=lr, clip_value=params['clipvalue'])
else:
    if ml_env.is_tpu is True:
        with tpu_strategy.scope():
            opti = tf.keras.optimizers.Adam(learning_rate=lr)
    else:
        opti = tf.keras.optimizers.Adam(learning_rate=lr)

if ml_env.is_tpu is True:
    with tpu_strategy.scope():
        model.compile(optimizer=opti, loss=loss, metrics=[], run_eagerly=False, jit_compile=True)
else:
    model.compile(optimizer=opti, loss=loss, metrics=['accuracy'])

In [24]:
import_checkpoint = True
force_import = False   # True: ignore metadata and try import anyway. This will of course crash, if the new model doesn't fit the checkpoint-data...

if import_checkpoint is True:
    import_previous_compatible_checkpoint(model, force_import=force_import)

Last checkpoint: /content/drive/My Drive/Colab Notebooks/women_writers/model/mhsa_v1_tf/training_checkpoints_8x(12, 12, 12, 12, 12, 12, 12, 12)x(512, 512, 512, 512, 512, 512, 512, 512)x20000/cp-16799.h5
Successful import of epoch 16823 from /content/drive/My Drive/Colab Notebooks/women_writers/model/mhsa_v1_tf/training_checkpoints_8x(12, 12, 12, 12, 12, 12, 12, 12)x(512, 512, 512, 512, 512, 512, 512, 512)x20000/cp-16799.h5, continuing from there...


In [25]:
model.summary()

Model: "mhsa_v1_tf"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 80)]              0         
                                                                 
 tf.cast (TFOpLambda)        (None, 80)                0         
                                                                 
 embedding (Embedding)       (None, 80, 128)           2560000   
                                                                 
 positional_encoding (Positi  (None, 80, 128)          0         
 onalEncoding)                                                   
                                                                 
 multi_head_self_attention (  (None, 80, 128)          3179008   
 MultiHeadSelfAttention)                                         
                                                                 
 multi_head_self_attention_1  (None, 80, 128)          3

In [26]:
TPU_GENERATE_ON_CPU = False  # The thing is: both options are slow on TPU :-/

class ServiceCallback(keras.callbacks.Callback):
#    def on_test_end(self, logs=None):
    # @tf.function
    def on_epoch_end(self, epoch, logs=None):
        save_model_metadata(epoch, suffix=model_suffix)
        if (epoch+1) % params['sample_every_n_epochs'] == 0:
            idx=random.randint(0,len(td)-1)
            text=td.decode(td[idx])
            print()
            if ml_env.is_tpu is True:
                temp_list=[0.7] # [0.6,0.7,0.8]
                gen_len=50
                with tpu_strategy.scope():
                    weights=model.get_weights()
                model_cpu.set_weights(weights)
                # HDF5 is required for saving weights that originate from TPU
                # otherwise this just silently fails...
                checkpoint_path = os.path.join(checkpoint_dir, "cp-{epoch:04d}.h5")
                chkpt_dest=checkpoint_path.format(epoch=epoch)
                print(f"Checkpoint: {chkpt_dest}")
                model_cpu.save_weights(chkpt_dest)
            else:
                temp_list=[0.6, 0.7, 0.8]
                gen_len=192
            print(f"prompt: {text}")
            for temp in temp_list:
                print(f"---------------- T={temp} ---------------")
                if ml_env.is_tpu is True and TPU_GENERATE_ON_CPU is True:
                    with tf.device('/cpu:0'):
                        if temp==0.0:
                            reply=mhsa_generate(model_cpu, text, gen_len=gen_len, temperature=1.0, argmax=True, verbose=False)
                        else:
                            reply=mhsa_generate(model_cpu, text, gen_len=gen_len, temperature=temp, verbose=False)
                else:
                    if temp==0.0:
                        reply=mhsa_generate(model_cpu, text, gen_len=gen_len, temperature=1.0, argmax=True, verbose=False)
                    else:
                        reply=mhsa_generate(model_cpu, text, gen_len=gen_len, temperature=temp, verbose=False)
                td.source_highlight(reply, min_quote_size=10, dark_mode=use_dark_mode, display_ref_anchor=False)
            print("--------------------------------------")

service_callback=ServiceCallback()

In [44]:
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

logdir = os.path.join(log_path, datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
if ml_env.is_tpu:
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, update_freq='epoch', write_graph=False)
else:
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, update_freq='batch')


In [45]:
# Dont try:
#    # use the python variable log_path:
#   get_ipython().run_line_magic('tensorboard', '--logdir "{log_path}"')
#except:
#   pass

# The following throws errors on non-colab, but the guarding above is too bug-ridden.
# if ml_env.is_tpu is False:
#    %tensorboard --logdir logs

## The actual training

In [46]:
EPOCHS=500000
if 'current_epoch' in params:
    initial_epoch=params['current_epoch']
else:
    initial_epoch=0

override=200
print(f"WARNING override of sample_every_n_epochs sample-generation to: {override}")
params['sample_every_n_epochs']=override

WARNING override of sample_every_n_epochs sample-generation to: 200


In [ ]:
if ml_env.is_tpu is True:
    steps_per_epoch=restricted_batches//params['batch_size']
    if steps_per_epoch < 1:
        steps_per_epoch = 1
    history = model.fit(dataset, epochs=EPOCHS, initial_epoch=initial_epoch, steps_per_epoch=steps_per_epoch, callbacks=[service_callback]) # for TPU we need to role our own checkpointer since we need to transfer the weights
else:
    history = model.fit(dataset, validation_data=validation_dataset, epochs=EPOCHS, initial_epoch=initial_epoch, callbacks=[checkpoint_callback, tensorboard_callback, service_callback])

Epoch 7945/500000
 6/23 [======>.......................] - ETA: 1s - loss: 0.2244

23/23 [==============================] - 104s 96ms/step - loss: 0.2203
Epoch 7946/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.2151
Epoch 7947/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.2165
Epoch 7948/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.2165
Epoch 7949/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.2100
Epoch 7950/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.2136
Epoch 7951/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.2085
Epoch 7952/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.2107
Epoch 7953/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.2117
Epoch 7954/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.2095
Epoch 7955/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.2082
Epoch 7956/500000
23/23 [==============================] - 

--------------------------------------
23/23 [==============================] - 48s 2s/step - loss: 0.2036
Epoch 8001/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.2061
Epoch 8002/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1997
Epoch 8003/500000
23/23 [==============================] - 2s 99ms/step - loss: 0.2029
Epoch 8004/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.2009
Epoch 8005/500000
23/23 [==============================] - 2s 99ms/step - loss: 0.2043
Epoch 8006/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.2048
Epoch 8007/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.2011
Epoch 8008/500000
23/23 [==============================] - 2s 99ms/step - loss: 0.2048
Epoch 8009/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.2010
Epoch 8010/500000
23/23 [==============================] - 2s 99ms/step - loss: 0.1997
Epoch 8011/500000
23/23

--------------------------------------
23/23 [==============================] - 49s 2s/step - loss: 0.1988
Epoch 8201/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.1996
Epoch 8202/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.2002
Epoch 8203/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.2017
Epoch 8204/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.2003
Epoch 8205/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.1914
Epoch 8206/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.1954
Epoch 8207/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1964
Epoch 8208/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1957
Epoch 8209/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1976
Epoch 8210/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1947
Epoch 8211/500000
23/23

--------------------------------------
23/23 [==============================] - 48s 2s/step - loss: 0.1952
Epoch 8401/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1987
Epoch 8402/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1958
Epoch 8403/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1942
Epoch 8404/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1951
Epoch 8405/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1973
Epoch 8406/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1947
Epoch 8407/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1989
Epoch 8408/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1988
Epoch 8409/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1933
Epoch 8410/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1964
Epoch 8411/500000
23/23

...

--------------------------------------
23/23 [==============================] - 49s 2s/step - loss: 0.1714
Epoch 16001/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1743
Epoch 16002/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1750
Epoch 16003/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1740
Epoch 16004/500000
23/23 [==============================] - 2s 99ms/step - loss: 0.1719
Epoch 16005/500000
23/23 [==============================] - 2s 99ms/step - loss: 0.1719
Epoch 16006/500000
23/23 [==============================] - 2s 99ms/step - loss: 0.1743
Epoch 16007/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1740
Epoch 16008/500000
23/23 [==============================] - 2s 99ms/step - loss: 0.1734
Epoch 16009/500000
23/23 [==============================] - 2s 99ms/step - loss: 0.1745
Epoch 16010/500000
23/23 [==============================] - 2s 99ms/step - loss: 0.1745
Epoch 16011/5

--------------------------------------
23/23 [==============================] - 48s 2s/step - loss: 0.1777
Epoch 16201/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1757
Epoch 16202/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1773
Epoch 16203/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1782
Epoch 16204/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1765
Epoch 16205/500000
23/23 [==============================] - 2s 99ms/step - loss: 0.1765
Epoch 16206/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1783
Epoch 16207/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1783
Epoch 16208/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1746
Epoch 16209/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1769
Epoch 16210/500000
23/23 [==============================] - 2s 99ms/step - loss: 0.1749
Epoch 16211/5

--------------------------------------
23/23 [==============================] - 49s 2s/step - loss: 0.1785
Epoch 16401/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1805
Epoch 16402/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1766
Epoch 16403/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1760
Epoch 16404/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.1772
Epoch 16405/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.1758
Epoch 16406/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1770
Epoch 16407/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1766
Epoch 16408/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1747
Epoch 16409/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1749
Epoch 16410/500000
23/23 [==============================] - 2s 98ms/step - loss: 0.1785
Epoch 16411/5

--------------------------------------
23/23 [==============================] - 48s 2s/step - loss: 0.1754
Epoch 16601/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.1742
Epoch 16602/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1746
Epoch 16603/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1745
Epoch 16604/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1763
Epoch 16605/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1737
Epoch 16606/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.1747
Epoch 16607/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.1790
Epoch 16608/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1756
Epoch 16609/500000
23/23 [==============================] - 2s 97ms/step - loss: 0.1750
Epoch 16610/500000
23/23 [==============================] - 2s 96ms/step - loss: 0.1760
Epoch 16611/5

--------------------------------------
23/23 [==============================] - 48s 2s/step - loss: 0.1723
Epoch 16801/500000
 8/23 [=========>....................] - ETA: 1s - loss: 0.1731

## A dialog with the trained model

In [27]:
model_cpu.set_weights(model.get_weights())

In [28]:
# Do a dialog with the recursive neural net trained above:
# def genDialogAnswer(prompt, g_state=None, endPrompt='.', maxEndPrompts=2,
# maxAnswerSize=512, temperature=1.0):

def doDialog(model):
    temperature = 0.6
    endPrompt = '.'  # the endPrompt character is the end-mark in answers.
    # look for number of maxEndPrompts until answer is finished.
    maxEndPrompts = 4
    maxAnswerSize = 2048  # Maximum length of the answer
    minAnswerSize = 64  # Minimum length of the answer
    print("Please enter some dialog.")
    print("The net will answer according to your input.")
    print("'bye' for end,")
    print("'reset' to reset the conversation context,")
    print("'temperature=<float>' [0.1(frozen)-1.0(creative)]")
    print("    to change character of the dialog.")
    print("    Current temperature={}.".format(temperature))
    print()
    xso = None
    bye = False
    doini = True
    bye = False
    while not bye:
        print("> ", end="")
        prompt = input()
        if prompt == 'bye':
            bye = True
            print("Good bye!")
            continue
        if prompt[:len("temperature=")] == "temperature=":
            t = float(prompt[len("temperature="):])
            if t > 0.05 and t < 1.4:
                temperature = t
                print("(generator temperature now {})".format(t))
                print()
                continue
            print("Invalid temperature-value ignored! [0.1-1.0]")
            continue
        reply=mhsa_generate(model, prompt, gen_len=256, temperature=temperature, verbose=True)
        td.source_highlight(reply, min_quote_size=13, dark_mode=use_dark_mode)

In [ ]:
# Talk to the net!
doDialog(model_cpu)

Please enter some dialog.
The net will answer according to your input.
'bye' for end,
'reset' to reset the conversation context,
'temperature=<float>' [0.1(frozen)-1.0(creative)]
    to change character of the dialog.
    Current temperature=0.6.

> It was a hot and warm summer evening, the stars were bright, and she was dreaming.
...... I’m come back with flame of St. John High, Stradly opened whether she spoke to her doing with white lay; but then he began to drawn Es—But yet said nothing but nothing else; she could possibly
this idea that she would forgive to
know not highly and more degence had
known to be five me as not the
reason to succepteder importance; they all music, many months left and to
Bertram’s opinion of Storhood, and what they
was the set, and to summon considerable of
dailious-bows of dark, or always
entirely more in his judgment, without the
cheerfully unluded in her own measure,
been recover of Mary-sipation, without any close by him with the most serious.

      

> 